### 问题1

In [159]:
from scipy.stats import norm
import math

p0 = 0.10  # 标称值10%
confidence_1 = 0.95  # 情形1: 95%信度
confidence_2 = 0.90  # 情形2: 90%信度

# 计算正态分布下的样本量
def calculate_sample_size_norm(p0, confidence):
    z = norm.ppf(confidence)  # 正态分布下的z值
    n = (z**2) * p0 * (1 - p0) / (0.05**2)  # 误差设为0.02
    return math.ceil(n)

# 计算样本量
sample_size_1 = calculate_sample_size_norm(p0, confidence_1)
sample_size_2 = calculate_sample_size_norm(p0, confidence_2)

sample_size_1, sample_size_2


(98, 60)

In [160]:
n_low=p0*(1-p0)/(0.05**2)
n_low

36.0

### 问题2

In [129]:
import random

n = 5000

def total_profit(n, part1, part2, part1_defect, part2_defect, part1_cost, part2_cost, part1_test_cost, part2_test_cost, product_defect_rate, assembly_cost, product_test_cost, market_price, replacement_loss, disassembly_cost, x1_1, x1_2, x2, x3):
    # 零件成本
    part_cost = n * part1_cost + n * part2_cost
    
    # 零件检测成本
    part_test_cost = x1_1 * n * part1_test_cost + x1_2 * n * part2_test_cost
    
    # 初次装配成本
    first_assembly_cost, product_q, after_part1, after_part2 = first_assemblycost(n, x1_1, x1_2, part1 , part2 , assembly_cost, part1_defect, part2_defect)

    # 对零件进行装配实际情况
    product_defect_q, assembly_defect, part1_defect_2, part2_defect_2, part1_success, part2_success = assembly(after_part1, after_part2, product_defect_rate)
    product_success_q = product_q - product_defect_q#直接发给客户的成品

    product_test_q, disassembly_q, re_assembly_q, replace_q,product_success_q = test_disassembly(x1_1, x1_2, x2, x3, after_part1, after_part2, product_defect_rate, product_q, product_defect_q, assembly_defect, part1_success, part2_success, part1_defect_2, part2_defect_2,product_success_q=product_success_q)

    # 成品检测成本
    total_product_test_cost = product_test_q * product_test_cost 

    # 拆解成本
    total_disassembly_cost = disassembly_q * disassembly_cost * x3

    # 再装配成本
    re_assembly_cost = re_assembly_q * assembly_cost * x3

    # 调换损失
    total_replacement_loss = replace_q * replacement_loss * (1-x2)

    # 销售额 = 成品数量 * 单价
    total_sale = product_success_q * market_price

    # 利润 = 销售额 - 零件成本 - 零件检测成本 - 初次装配成本 - 成品检测成本 - 拆解成本 - 再装配成本 - 调换损失
    profit = total_sale - part_cost - part_test_cost - first_assembly_cost - total_product_test_cost - total_disassembly_cost - re_assembly_cost - total_replacement_loss

    return profit

#返回 初次装配成本、第一次装配成品数量、检测后的零件列表
def first_assemblycost(n, x1_1, x1_2, part1 , part2 , assembly_cost, part1_defect, part2_defect):
    if x1_1 == 1 and x1_2 == 0:#零件1检测，零件2不检测
        product_q = n - part1_defect
        part1 = [1] * (n - part1_defect)
        part2 = part2
        total_assembly_cost = (n - part1_defect) * assembly_cost
    elif x1_1 == 0 and x1_2 == 1:#零件1不检测，零件2检测
        product_q = n - part2_defect
        part1 = part1
        part2 = [1] * (n - part2_defect)
        total_assembly_cost = (n - part2_defect) * assembly_cost
    elif x1_1 == 1 and x1_2 == 1:#两零件均检测
        product_q = min(n - part1_defect, n - part2_defect)
        part1 = [1] * (n - part1_defect)
        part2 = [1] * (n - part2_defect)
        total_assembly_cost = product_q * assembly_cost
    else:#两零件均不检测
        product_q = n
        part1 = part1
        part2 = part2
        total_assembly_cost = n * assembly_cost
    return total_assembly_cost, product_q, part1, part2

#对装配过程进行模拟，返回 每次装配 总次品数量、零件12不合格数量、零件不合格导致成品不合格中零件12合格的数量
def assembly(part1, part2, product_defect_rate):
    product_success_q = 0#记录每次装配两零件均合格的成品数量
    product_defect_q = 0
    part1_defect_2 = 0#每次装配，零件1不合格的数量
    part2_defect_2 = 0#每次装配，零件2不合格的数量
    part1_success = 0#每次装配，一个零件不合格的成品中，零件1合格的数量
    part2_success = 0#每次装配，一个零件不合格的成品中，零件2合格的数量
    for i in range(min(len(part1), len(part2))):
        if part1[i] == 1 and part2[i] == 1:
            product_success_q += 1
        elif part1[i] == 1 and part2[i] == 0:
            product_defect_q += 1
            part1_success += 1
            part2_defect_2 += 1
        elif part1[i] == 0 and part2[i] == 1:
            product_defect_q += 1
            part1_defect_2 += 1
            part2_success += 1
        else:
            product_defect_q += 1
            part1_defect_2 += 1
            part2_defect_2 += 1
    assembly_defect = int(product_success_q * product_defect_rate)#每次装配导致的次品数量
    product_defect_q += assembly_defect#每次装配总次品数量
    return product_defect_q, assembly_defect, part1_defect_2, part2_defect_2, part1_success, part2_success

#对拆解过程进行模拟，更新零件列表
def disassembly(x1_1, x1_2, part1, part2, assembly_defect, part1_success, part2_success, part1_defect_2, part2_defect_2):
    if x1_1 == 0 and x1_2 == 0:
        new_part1 = [1] * (assembly_defect + part1_success) + [0] * part1_defect_2
        new_part2 = [1] * (assembly_defect + part2_success) + [0] * part2_defect_2
    elif x1_1 == 0 and x1_2 == 1:
        new_part1 = [1] * assembly_defect + [0] * part1_defect_2 + part1[len(part2):]
        new_part2 = [1] * (assembly_defect + part2_success)
    elif x1_1 == 1 and x1_2 == 0:
        new_part1 = [1] * (assembly_defect + part1_success)
        new_part2 = [1] * assembly_defect + [0] * part2_defect_2 + part2[len(part1):]
    else:
        new_part1 = [1] * assembly_defect
        new_part2 = [1] * assembly_defect
    random.shuffle(new_part1)
    random.shuffle(new_part2)
    return new_part1, new_part2

#模拟对成品检测、拆解
def test_disassembly(x1_1, x1_2, x2, x3, part1, part2, product_defect_rate, product_q, product_defect_q, assembly_defect, part1_success, part2_success, part1_defect_2, part2_defect_2,product_success_q):
    disassembly_q = product_defect_q#若拆卸，初始化拆卸总数量
    re_assembly_q = product_defect_q#若拆卸，初始化重新装配总数量
    if x2:  # 检测
        product_test_q = min(len(part1),len(part2))#若检测，初始化检测总数量
        replace_q = 0
        if x3:  # 拆解
            for i in range(3):
                new_part1, new_part2 = disassembly(x1_1, x1_2, part1, part2, assembly_defect, part1_success, part2_success, part1_defect_2, part2_defect_2)
                re_assembly_q += min(len(new_part1), len(new_part2))#累加每次拆解后，再装配数量
                product_test_q += min(len(new_part1), len(new_part2))#累加每次拆解后，再检测数量
                product_defect_q, assembly_defect, part1_defect_2, part2_defect_2, part1_success, part2_success = assembly(new_part1, new_part2, product_defect_rate)
                disassembly_q += product_defect_q#累加每次拆卸的成品数量
                product_success_q+=(min(len(new_part1), len(new_part2))-product_defect_q)#累加每次重新装配中，可以直接发给客户的产品数量
        else:  # 不拆解
            product_test_q = product_test_q#总检测数量不变，不新增销售数量
    else:  # 不检测
        product_test_q = 0
        replace_q = product_defect_q#若不检测，初始化客户调换数量
        if x3:  # 拆解
            for i in range(3):
                new_part1, new_part2 = disassembly(x1_1, x1_2, part1, part2, assembly_defect, part1_success, part2_success, part1_defect_2, part2_defect_2)
                re_assembly_q += min(len(new_part1), len(new_part2))#累加每次拆解后，再装配数量
                product_defect_q, assembly_defect, part1_defect_2, part2_defect_2, part1_success, part2_success = assembly(new_part1, new_part2, product_defect_rate)
                disassembly_q += product_defect_q#累加每次拆卸的成品数量
                replace_q += product_defect_q#累加每次装配后，客户调换数量
                product_success_q+=min(len(new_part1), len(new_part2))-product_defect_q
        else:  # 不拆解
            replace_q = product_defect_q#总退换数量不变，不新增销售数量
    return product_test_q, disassembly_q, re_assembly_q, replace_q,product_success_q
# 测试数据
situations = [
    {"part1_defect_rate": 0.1, "part1_cost": 4, "part1_test_cost": 2,
     "part2_defect_rate": 0.1, "part2_cost": 18, "part2_test_cost": 3,
     "product_defect_rate": 0.1, "assembly_cost": 6, "product_test_cost": 3,
     "market_price": 56, "replacement_loss": 6, "disassembly_cost": 5},

    {"part1_defect_rate": 0.2, "part1_cost": 4, "part1_test_cost": 2,
     "part2_defect_rate": 0.2, "part2_cost": 18, "part2_test_cost": 3,
     "product_defect_rate": 0.2, "assembly_cost": 6, "product_test_cost": 3,
     "market_price": 56, "replacement_loss": 6, "disassembly_cost": 5},

    {"part1_defect_rate": 0.1, "part1_cost": 4, "part1_test_cost": 2,
     "part2_defect_rate": 0.1, "part2_cost": 18, "part2_test_cost": 3,
     "product_defect_rate": 0.1, "assembly_cost": 6, "product_test_cost": 3,
     "market_price": 56, "replacement_loss": 30, "disassembly_cost": 5},

    {"part1_defect_rate": 0.2, "part1_cost": 4, "part1_test_cost": 1,
     "part2_defect_rate": 0.2, "part2_cost": 18, "part2_test_cost": 1,
     "product_defect_rate": 0.2, "assembly_cost": 6, "product_test_cost": 2,
     "market_price": 56, "replacement_loss": 30, "disassembly_cost": 5},

    {"part1_defect_rate": 0.1, "part1_cost": 4, "part1_test_cost": 8,
     "part2_defect_rate": 0.2, "part2_cost": 18, "part2_test_cost": 1,
     "product_defect_rate": 0.1, "assembly_cost": 6, "product_test_cost": 2,
     "market_price": 56, "replacement_loss": 10, "disassembly_cost": 5},

    {"part1_defect_rate": 0.05, "part1_cost": 4, "part1_test_cost": 2,
     "part2_defect_rate": 0.05, "part2_cost": 18, "part2_test_cost": 3,
     "product_defect_rate": 0.05, "assembly_cost": 6, "product_test_cost": 3,
     "market_price": 56, "replacement_loss": 10, "disassembly_cost": 40}
]

for situation in situations:
    part1_defect = int(n * situation['part1_defect_rate'])
    part2_defect = int(n * situation['part2_defect_rate'])

    part1 = [1] * (n - part1_defect) + [0] * part1_defect
    part2 = [1] * (n - part2_defect) + [0] * part2_defect
    
    random.shuffle(part1)
    random.shuffle(part2)

    max_profit = float('-inf')
    best_strategy = None

    for x1_1 in range(2):
        for x1_2 in range(2):
            for x2 in range(2):
                for x3 in range(2):
                    profit = total_profit(n, part1, part2, part1_defect, part2_defect, 
                                          situation['part1_cost'], situation['part2_cost'],
                                          situation['part1_test_cost'], situation['part2_test_cost'],
                                          situation['product_defect_rate'], situation['assembly_cost'],
                                          situation['product_test_cost'], situation['market_price'],
                                          situation['replacement_loss'], situation['disassembly_cost'],
                                          x1_1, x1_2, x2, x3)
                    if profit > max_profit:
                        max_profit = profit
                        best_strategy = (x1_1, x1_2, x2, x3)

    print(f"最大利润: {max_profit}")
    print(f"最佳策略: 检测零件1: {'是' if best_strategy[0] else '否'}, " 
          f"检测零件2: {'是' if best_strategy[1] else '否'}, "
          f"检测成品: {'是' if best_strategy[2] else '否'}, "
          f"拆解不合格品: {'是' if best_strategy[3] else '否'}")

最大利润: 78817
最佳策略: 检测零件1: 是, 检测零件2: 是, 检测成品: 否, 拆解不合格品: 是
最大利润: 42934
最佳策略: 检测零件1: 是, 检测零件2: 是, 检测成品: 否, 拆解不合格品: 是
最大利润: 66841
最佳策略: 检测零件1: 是, 检测零件2: 是, 检测成品: 否, 拆解不合格品: 是
最大利润: 53938
最佳策略: 检测零件1: 是, 检测零件2: 是, 检测成品: 是, 拆解不合格品: 是
最大利润: 57010
最佳策略: 检测零件1: 否, 检测零件2: 是, 检测成品: 是, 拆解不合格品: 是
最大利润: 92876
最佳策略: 检测零件1: 否, 检测零件2: 否, 检测成品: 否, 拆解不合格品: 否


### 问题3

In [153]:
import random
import itertools
#模拟函数
#输出：销售量、新增销售量、调换总量、半成品拆解总量、成品拆解总量、半成品再检测总量、成品再检测总量

#1.零件的检测   更新零件列表，得到半成品数量
#2.零件组装为半成品  得到ok的半成品数量、两个零件都不合格的半成品数量、零件1合格2不合格半成品数量、零件1不合格2合格半成品数量、装配不合格的半成品数量
#3.半成品的检测与拆解  得到半成品检测总量、半成品拆解总量、新增销售量
#4.半成品组装为成品 得到ok的半成品数量、两个零件都不合格的半成品数量、零件1合格2不合格半成品数量、零件1不合格2合格半成品数量、装配不合格的半成品数量
#5.成品的检测与拆解 得到半成品检测总量、半成品拆解总量、新增销售量

#1.1零件的检测(两个零件)
#输出 第一次装配成品数量、检测后的零件列表
def first_assemblycost_1(n, x1_1, x1_2, part1 , part2 , part1_defect, part2_defect):
    if x1_1 == 1 and x1_2 == 0:#零件1检测，零件2不检测
        product_q = n - part1_defect
        part1 = [1] * (n - part1_defect)
        part2 = part2
    elif x1_1 == 0 and x1_2 == 1:#零件1不检测，零件2检测
        product_q = n - part2_defect
        part1 = part1
        part2 = [1] * (n - part2_defect)
    elif x1_1 == 1 and x1_2 == 1:#两零件均检测
        product_q = min(n - part1_defect, n - part2_defect)
        part1 = [1] * (n - part1_defect)
        part2 = [1] * (n - part2_defect)
    else:#两零件均不检测
        product_q = n
        part1 = part1
        part2 = part2
    return product_q, part1, part2

#2.1零件组装为半成品(2组1)
#得到 次品总数量,合格品总量
def assembly_1(part1, part2, product_defect_rate):
    product_success_q = 0#记录每次装配两零件均合格的成品数量
    product_defect_q = 0#每次装配总次品数量
    for i in range(min(len(part1), len(part2))):
        if part1[i] == 1 and part2[i] == 1:
            product_success_q += 1
        else:
            product_defect_q += 1
    assembly_defect = int(product_success_q * product_defect_rate)#每次装配导致的次品数量
    product_defect_q += assembly_defect#每次装配总次品数量
    product_success_q = product_success_q - assembly_defect#完全ok的成品
    return product_defect_q, product_success_q,assembly_defect


#拆解函数(拆成两个)
#输出:拆解得到的零件列表
def disassembly_1(x1_1, x1_2, part1, part2, part1_defect , part2_defect , assembly_defect , product_defect_q):
    if x1_1 == 0 and x1_2 == 0:
        new_part1 = [1] * (product_defect_q-part1_defect) + [0] * part1_defect
        new_part2 = [1] * (product_defect_q-part2_defect) + [0] * part2_defect
    elif x1_1 == 0 and x1_2 == 1:
        new_part1 = [1] * assembly_defect + [0] * (product_defect_q-assembly_defect) + part1[len(part2):]
        new_part2 = [1] * product_defect_q
    elif x1_1 == 1 and x1_2 == 0:
        new_part1 = [1] * product_defect_q
        new_part2 = [1] * assembly_defect + [0] * (product_defect_q-assembly_defect) + part2[len(part1):]
    else:
        new_part1 = [1] * assembly_defect
        new_part2 = [1] * assembly_defect
    random.shuffle(new_part1)
    random.shuffle(new_part2)
    return new_part1, new_part2

#检测拆解(拆成两个)
#模拟对成品检测、拆解
def test_disassembly_1(x1_1, x1_2, x2, x3, part1, part2, part1_defect,part2_defect,product_defect_rate, product_defect_q, assembly_defect,product_success_q):
    disassembly_q = product_defect_q#若拆卸，初始化拆卸总数量
    re_assembly_q = 0#若拆卸，初始化重新装配总数量
    if x2:  # 检测
        product_test_q = min(len(part1),len(part2))#若检测，初始化检测总数量
        replace_q = 0
        if x3:  # 拆解
            for i in range(3):
                new_part1, new_part2 = disassembly_1(x1_1, x1_2, part1, part2, part1_defect , part2_defect , assembly_defect , product_defect_q)
                re_assembly_q += min(len(new_part1), len(new_part2))#累加每次拆解后，再装配数量
                product_test_q += min(len(new_part1), len(new_part2))#累加每次拆解后，再检测数量
                product_defect_q, product_success_q,_ = assembly_1(part1, part2, product_defect_rate)
                disassembly_q += product_defect_q#累加每次拆卸的成品数量
                product_success_q+=(min(len(new_part1), len(new_part2))-product_defect_q)#累加每次重新装配中，可以直接发给客户的产品数量
        else:  # 不拆解
            product_test_q = product_test_q#总检测数量不变，不新增销售数量
    else:  # 不检测
        product_test_q = 0
        replace_q = product_defect_q#若不检测，初始化客户调换数量
        if x3:  # 拆解
            for i in range(3):
                new_part1, new_part2 = disassembly_1(x1_1, x1_2, part1, part2, part1_defect , part2_defect , assembly_defect , product_defect_q)
                re_assembly_q += min(len(new_part1), len(new_part2))#累加每次拆解后，再装配数量
                product_defect_q, product_success_q,_ = assembly_1(part1, part2, product_defect_rate)
                disassembly_q += product_defect_q#累加每次拆卸的成品数量
                replace_q += product_defect_q#累加每次装配后，客户调换数量
                product_success_q+=min(len(new_part1), len(new_part2))-product_defect_q
        else:  # 不拆解
            replace_q = product_defect_q#总退换数量不变，不新增销售数量
    return product_test_q, disassembly_q, re_assembly_q, replace_q,product_success_q





#对于三组一

#1.2零件的检测(三个零件)
#输出 第一次装配成品数量、检测后的零件列表
def first_assemblycost_2(n, x1_1, x1_2 , x1_3 , part1 , part2 , part3 , part1_defect, part2_defect , part3_defect):
    part1 = part1
    part2 = part2
    part3 = part3
    if x1_1 == 1 :#零件1检测
        part1 = [1] * (n - part1_defect)
    elif x1_2 == 1:#零件2检测
        part2 = [1] * (n - part2_defect)
    elif x1_3 == 1:#零件3检测
        part3 = [1] * (n - part3_defect)
    else:#两零件均不检测
        part1 = part1
        part2 = part2
        part3 = part3
    product_q = min(len(part1),len(part2),len(part3))
    return product_q, part1, part2 , part3

#2.2（3组1）
def assembly_2(part1 , part2 , part3 , product_defect_rate):
    product_success_q=0
    product_defect_q=0
    part1_defect_2=0
    part2_defect_2=0
    part3_defect_2=0
    for i in range(min(len(part1),len(part2),len(part3))):
        if part1[i] == 1 and part2[i] == 1 and part3[i] == 1:
            product_success_q+=1
        else:
            part1_defect_2+=1*(1-part1[i])
            part2_defect_2+=1*(1-part2[i])
            part3_defect_2+=1*(1-part3[i])#记录每种次品零件进入下一步的数量
            product_defect_q+=1
    assembly_defect=int(product_success_q*product_defect_rate)
    product_defect_q += assembly_defect
    product_success_q = product_success_q-assembly_defect
    return product_defect_q,product_success_q,part1_defect_2,part2_defect_2,part3_defect_2

#拆解函数(拆成三个)
#输入零件检测更新后的零件列表
#输出:拆解得到的零件列表
def disassembly_2(x1_1, x1_2,x1_3, part1, part2,part3, product_defect_q , part1_defect_2 , part2_defect_2 , part3_defect_2):
    if x1_1 == 0 and x1_2 == 0 and x1_3 == 0:
        new_part1 = [1] * (product_defect_q-part1_defect_2) + [0] * part1_defect_2
        new_part2 = [1] * (product_defect_q-part2_defect_2) + [0] * part2_defect_2
        new_part3 = [1] * (product_defect_q-part3_defect_2) + [0] * part3_defect_2
    elif x1_1 == 1 and x1_2 == 0 and x1_3 == 0:
        new_part1 = [1] * product_defect_q
        new_part2 = [1] * (product_defect_q-part2_defect_2) + [0] * part2_defect_2 + part2[len(part1):]
        new_part3 = [1] * (product_defect_q-part3_defect_2) + [0] * part3_defect_2 + part3[len(part1):]
    elif x1_1 == 0 and x1_2 == 1 and x1_3 == 0:
        new_part1 = [1] * (product_defect_q-part2_defect_2) + [0] * part1_defect_2 + part1[len(part2):]
        new_part2 = [1] * product_defect_q
        new_part3 = [1] * (product_defect_q-part2_defect_2) + [0] * part3_defect_2 + part3[len(part2):]
    elif x1_1 == 0 and x1_2 == 0 and x1_3 == 1:
        new_part1 = [1] * (product_defect_q-part2_defect_2) + [0] * part2_defect_2 + part2[len(part3):]
        new_part2 = [1] * (product_defect_q-part2_defect_2) + [0] * part3_defect_2 + part3[len(part3):]
        new_part3 = [1] * product_defect_q
    elif x1_1 == 1 and x1_2 == 1 and x1_3 == 0:
        new_part1 = [1] * product_defect_q
        new_part2 = [1] * product_defect_q
        new_part3 = [1] * (product_defect_q-part2_defect_2) + [0] * part3_defect_2
    elif x1_1 == 1 and x1_2 == 0 and x1_3 == 1:
        new_part1 = [1] * product_defect_q
        new_part2 = [1] * (product_defect_q-part2_defect_2) + [0] * part2_defect_2
        new_part3 = [1] * product_defect_q
    elif x1_1 == 0 and x1_2 == 1 and x1_3 == 1:
        new_part1 = [1] * (product_defect_q-part2_defect_2) + [0] * part1_defect_2
        new_part2 = [1] * product_defect_q
        new_part3 = [1] * product_defect_q
    else:
        new_part1 = [1] * product_defect_q
        new_part2 = [1] * product_defect_q
        new_part3 = [1] * product_defect_q
    random.shuffle(new_part1)
    random.shuffle(new_part2)
    return new_part1, new_part2 , new_part3

#检测拆卸(拆成三个)
def test_disassembly_2(x1_1, x1_2,x1_3, x2, x3, part1, part2, part3 ,product_defect_rate, product_defect_q, product_success_q,part1_defect_2,part2_defect_2,part3_defect_2):
    disassembly_q = product_defect_q#若拆卸，初始化拆卸总数量
    re_assembly_q = 0#若拆卸，初始化重新装配总数量
    re_product_success_q=product_success_q#初始化合格产品
    if x2:  # 检测
        product_test_q = min(len(part1),len(part2),len(part3))#若检测，初始化检测总数量
        replace_q = 0
        if x3:  # 拆解
            for i in range(3):
                new_part1, new_part2, new_part3 = disassembly_2(x1_1, x1_2,x1_3, part1, part2,part3 , product_defect_q , part1_defect_2 , part2_defect_2 , part3_defect_2)
                re_assembly_q += min(len(new_part1), len(new_part2),len(new_part3))#累加每次拆解后，再装配数量
                product_test_q += min(len(new_part1), len(new_part2),len(new_part3))#累加每次拆解后，再检测数量
                product_defect_q,product_success_q,part1_defect_2,part2_defect_2,part3_defect_2= assembly_2(new_part1, new_part2, new_part3 , product_defect_rate)
                disassembly_q += product_defect_q#累加每次拆卸的成品数量
                re_product_success_q+=product_success_q#累加每次重新装配中，可以直接发给客户的产品数量
        else:  # 不拆解
            product_test_q = product_test_q#总检测数量不变，不新增销售数量
    else:  # 不检测
        product_test_q = 0
        replace_q = product_defect_q#若不检测，初始化客户调换数量
        if x3:  # 拆解
            for i in range(3):
                new_part1, new_part2,new_part3 = disassembly_2(x1_1, x1_2,x1_3, part1, part2,part3 , product_defect_q , part1_defect_2 , part2_defect_2 , part3_defect_2)
                re_assembly_q += min(len(new_part1), len(new_part2),len(new_part3))#累加每次拆解后，再装配数量
                product_defect_q,product_success_q,part1_defect_2,part2_defect_2,part3_defect_2= assembly_2(new_part1, new_part2, new_part3 , product_defect_rate)
                disassembly_q += product_defect_q#累加每次拆卸的成品数量
                replace_q += product_defect_q#累加每次装配后，客户调换数量(进入半成品的不合格半成品)
                re_product_success_q+=product_success_q
        else:  # 不拆解
            replace_q = product_defect_q#总退换数量不变，不新增销售数量
    return product_test_q, disassembly_q, re_assembly_q, replace_q,re_product_success_q


def total_profit(n,x1_1,x1_2,x1_3,x1_4,x1_5,x1_6,x1_7,x1_8,x1_9,x1_10,x1_11,x1_12,x1_13,x1_14,x2,x3,part1,part2,part3,part4,part5,part6,part7,part8,part1_defect,part2_defect,part3_defect,part4_defect,part5_defect,part6_defect,part7_defect,part8_defect):
    product_defect_rate=0.1#次品率
    product_test_cost_1=4#半成品检测成本
    product_test_cost_2=6#成品检测成本
    disassembly_cost_1=6#半成品拆解成本
    disassembly_cost_2=10#成品拆解成本
    assembly_cost=8#装配成本
    replacement_loss=40#调换损失
    market_price=200#市场售价

    # 零件成本
    part_cost = n * 64
    
    # 零件检测成本
    part_test_cost = n*(x1_1+x1_2+2*x1_3+x1_4+x1_5+2*x1_6+x1_7+2*x1_8)
    
    # 2组1零件检测
    product_q_3, part7, part8 = first_assemblycost_1(n, x1_7, x1_8, part7 , part8 , part7_defect, part8_defect)

    #3组1零件检测
    product_q_1,part1,part2,part3=first_assemblycost_2(n,x1_1,x1_2,x1_3,part1,part2,part3,part1_defect,part2_defect,part3_defect)
    product_q_2,part4,part5,part6=first_assemblycost_2(n,x1_4,x1_5,x1_6,part4,part5,part6,part4_defect,part5_defect,part6_defect)

    # 对零件进行装配实际情况
    product_defect_q_3,product_success_q_3,assembly_defect_3 = assembly_1(part1,part2 ,product_defect_rate)#半成品3不合格数量
    product_defect_q_1,product_success_q_1,_,_,_ = assembly_2(part1,part2,part3, product_defect_rate)#半成品1不合格数量
    product_defect_q_2,product_success_q_2,_,_,_ = assembly_2(part4,part5,part6, product_defect_rate)#半成品2不合格数量

    product_test_q_3, disassembly_q_3, re_assembly_q_3, replace_q_3,product_success_q_3 = test_disassembly_1(x1_7, x1_8, x1_11, x1_14, part7, part8, part7_defect,part8_defect,product_defect_rate, product_defect_q_3,  assembly_defect_3, product_success_q_3)
    product_test_q_1, disassembly_q_1, re_assembly_q_1, replace_q_1,product_success_q_1 = test_disassembly_2(x1_1, x1_2,x1_3, x1_9, x1_12, part1, part2, part3, product_defect_rate, product_defect_q_1,product_success_q_1,part1_defect,part2_defect,part3_defect)
    product_test_q_2, disassembly_q_2, re_assembly_q_2, replace_q_2,product_success_q_2 = test_disassembly_2(x1_4, x1_5,x1_6, x1_10, x1_13, part1, part2, part4, product_defect_rate, product_defect_q_2,product_success_q_2,part4_defect,part5_defect,part6_defect,)

    #生成半成品列表
    part9=[1]*product_success_q_1+[0]*(product_q_1-product_success_q_1)
    part10=[1]*product_success_q_2+[0]*(product_q_2-product_success_q_2)
    part11=[1]*product_success_q_3+[0]*(product_q_3-product_success_q_3)
    random.shuffle(part9)
    random.shuffle(part10)
    random.shuffle(part11)

    #半成品装配为成品
    product_defect_q_4,product_success_q_4,_,_,_ = assembly_2(part9,part10,part11, product_defect_rate)#成品不合格数量

    product_test_q_4, disassembly_q_4, re_assembly_q_4, replace_q_4,product_success_q_4 = test_disassembly_2(x1_9, x1_10,x1_11, x2, x3, part9, part10, part11, product_defect_rate, product_defect_q_4,product_success_q_4,product_q_1-product_success_q_1,product_q_2-product_success_q_2,product_q_3-product_success_q_3)

    # 成品检测成本
    total_product_test_cost = (product_test_q_1 * x1_9+product_test_q_2*x1_10+product_test_q_3*x1_11)* product_test_cost_1+product_test_q_4*x3*product_test_cost_2

    # 拆解成本
    total_disassembly_cost = (disassembly_q_1 * x1_12+disassembly_q_2 * x1_13+disassembly_q_3 * x1_14)* disassembly_cost_1+disassembly_q_4*x3*disassembly_cost_2

    # 装配成本
    total_assembly_cost=(product_q_1+product_q_2+product_q_3+re_assembly_q_1+re_assembly_q_2+re_assembly_q_3+re_assembly_q_4)*assembly_cost

    # 调换损失
    total_replacement_loss = replace_q_4 * replacement_loss * (1-x2)

    # 销售额 = 成品数量 * 单价
    total_sale = product_success_q_4* market_price

    # 利润 = 销售额 - 零件成本 - 零件检测成本  - 成品检测成本 - 拆解成本 - 装配成本 - 调换损失
    profit = total_sale - part_cost - part_test_cost  - total_product_test_cost - total_disassembly_cost - total_assembly_cost - total_replacement_loss

    return profit
parts = [
    {'defect_rate': 0.1, 'cost': 2, 'test_cost': 1},
    {'defect_rate': 0.1, 'cost': 8, 'test_cost': 1},
    {'defect_rate': 0.1, 'cost': 12, 'test_cost': 2},
    {'defect_rate': 0.1, 'cost': 2, 'test_cost': 1},
    {'defect_rate': 0.1, 'cost': 8, 'test_cost': 1},
    {'defect_rate': 0.1, 'cost': 12, 'test_cost': 2},
    {'defect_rate': 0.1, 'cost': 8, 'test_cost': 1},
    {'defect_rate': 0.1, 'cost': 12, 'test_cost': 2}
]

n=5000
part1_defect_q = int(n * 0.1)
part2_defect_q = int(n * 0.1)
part3_defect_q = int(n * 0.1)
part4_defect_q = int(n * 0.1)
part5_defect_q = int(n * 0.1)
part6_defect_q = int(n * 0.1)
part7_defect_q = int(n * 0.1)
part8_defect_q = int(n * 0.1)

part1 = [1] * (n - part1_defect_q) + [0] * part1_defect_q
part2 = [1] * (n - part2_defect_q) + [0] * part2_defect_q
part3 = [1] * (n - part3_defect_q) + [0] * part3_defect_q
part4 = [1] * (n - part4_defect_q) + [0] * part4_defect_q
part5 = [1] * (n - part5_defect_q) + [0] * part5_defect_q
part6 = [1] * (n - part6_defect_q) + [0] * part6_defect_q
part7 = [1] * (n - part7_defect_q) + [0] * part7_defect_q
part8 = [1] * (n - part8_defect_q) + [0] * part8_defect_q
    
random.shuffle(part1)
random.shuffle(part2)
random.shuffle(part3)
random.shuffle(part4)
random.shuffle(part5)
random.shuffle(part6)
random.shuffle(part7)
random.shuffle(part8)

max_profit = float('-inf')
best_strategy = None
for x1 in itertools.product([0, 1], repeat=14):
    for x2 in range(2):
        for x3 in range(2):
            profit = total_profit(n,x1[0],x1[1],x1[2],x1[3],x1[4],x1[5],x1[6],x1[7],x1[8],x1[9],x1[10],x1[11],x1[12],x1[13],x2,x3, part1,part2,part3,part4,part5,part6,part7,part8,part1_defect_q,part2_defect_q,part3_defect_q,part4_defect_q,part5_defect_q,part6_defect_q,part7_defect_q,part8_defect_q)
            if profit > max_profit:
                max_profit = profit
                best_strategy = (x1, x2, x3)

In [156]:
print(f"最大利润: {max_profit}")
print(f"最佳策略: 检测零件1: {'是' if best_strategy[0] else '否'}, " ,
        f"检测零件2: {'是' if best_strategy[0][1] else '否'}, ",
        f"检测零件3: {'是' if best_strategy[0][2] else '否'}, ",
        f"检测零件4: {'是' if best_strategy[0][3] else '否'}, ",
        f"检测零件5: {'是' if best_strategy[0][4] else '否'}, ",
        f"检测零件6: {'是' if best_strategy[0][5] else '否'}, ",
        f"检测零件7: {'是' if best_strategy[0][6] else '否'}, ",
        f"检测零件8: {'是' if best_strategy[0][7] else '否'}, ",
        f"检测半成品1: {'是' if best_strategy[0][8] else '否'}, ",
        f"检测半成品1: {'是' if best_strategy[0][9] else '否'}, ",
        f"检测半成品1: {'是' if best_strategy[0][10] else '否'}, ",
        f"拆解不半成品1: {'是' if best_strategy[0][11] else '否'}",
        f"拆解不半成品2: {'是' if best_strategy[0][12] else '否'}",
        f"拆解不半成品3: {'是' if best_strategy[0][13] else '否'}",
        f"检测成品: {'是' if best_strategy[1] else '否'}, ",
        f"拆解成品: {'是' if best_strategy[2] else '否'}",sep='\n')

最大利润: 373178
最佳策略: 检测零件1: 是, 
检测零件2: 否, 
检测零件3: 否, 
检测零件4: 否, 
检测零件5: 否, 
检测零件6: 否, 
检测零件7: 否, 
检测零件8: 否, 
检测半成品1: 是, 
检测半成品1: 是, 
检测半成品1: 是, 
拆解不半成品1: 否
拆解不半成品2: 否
拆解不半成品3: 否
检测成品: 是, 
拆解成品: 是


### 问题四

In [161]:
import random
import numpy as np

def total_profit(n, part1, part2, part1_defect, part2_defect, part1_cost, part2_cost, part1_test_cost, part2_test_cost, product_defect_rate, assembly_cost, product_test_cost, market_price, replacement_loss, disassembly_cost, x1_1, x1_2, x2, x3):
    # 零件成本
    part_cost = n * part1_cost + n * part2_cost
    
    # 零件检测成本
    part_test_cost = x1_1 * n * part1_test_cost + x1_2 * n * part2_test_cost
    
    # 初次装配成本
    first_assembly_cost, product_q, after_part1, after_part2 = first_assemblycost(n, x1_1, x1_2, part1 , part2 , assembly_cost, part1_defect, part2_defect)

    # 对零件进行装配实际情况
    product_defect_q, assembly_defect, part1_defect_2, part2_defect_2, part1_success, part2_success = assembly(after_part1, after_part2, product_defect_rate)
    product_success_q = product_q - product_defect_q#直接发给客户的成品

    product_test_q, disassembly_q, re_assembly_q, replace_q,product_success_q = test_disassembly(x1_1, x1_2, x2, x3, after_part1, after_part2, product_defect_rate, product_q, product_defect_q, assembly_defect, part1_success, part2_success, part1_defect_2, part2_defect_2,product_success_q=product_success_q)

    # 成品检测成本
    total_product_test_cost = product_test_q * product_test_cost 

    # 拆解成本
    total_disassembly_cost = disassembly_q * disassembly_cost * x3

    # 再装配成本
    re_assembly_cost = re_assembly_q * assembly_cost * x3

    # 调换损失
    total_replacement_loss = replace_q * replacement_loss * (1-x2)

    # 销售额 = 成品数量 * 单价
    total_sale = product_success_q * market_price

    # 利润 = 销售额 - 零件成本 - 零件检测成本 - 初次装配成本 - 成品检测成本 - 拆解成本 - 再装配成本 - 调换损失
    profit = total_sale - part_cost - part_test_cost - first_assembly_cost - total_product_test_cost - total_disassembly_cost - re_assembly_cost - total_replacement_loss

    return profit

#返回 初次装配成本、第一次装配成品数量、检测后的零件列表
def first_assemblycost(n, x1_1, x1_2, part1 , part2 , assembly_cost, part1_defect, part2_defect):
    if x1_1 == 1 and x1_2 == 0:#零件1检测，零件2不检测
        product_q = n - part1_defect
        part1 = [1] * (n - part1_defect)
        part2 = part2
        total_assembly_cost = (n - part1_defect) * assembly_cost
    elif x1_1 == 0 and x1_2 == 1:#零件1不检测，零件2检测
        product_q = n - part2_defect
        part1 = part1
        part2 = [1] * (n - part2_defect)
        total_assembly_cost = (n - part2_defect) * assembly_cost
    elif x1_1 == 1 and x1_2 == 1:#两零件均检测
        product_q = min(n - part1_defect, n - part2_defect)
        part1 = [1] * (n - part1_defect)
        part2 = [1] * (n - part2_defect)
        total_assembly_cost = product_q * assembly_cost
    else:#两零件均不检测
        product_q = n
        part1 = part1
        part2 = part2
        total_assembly_cost = n * assembly_cost
    return total_assembly_cost, product_q, part1, part2

#对装配过程进行模拟，返回 每次装配 总次品数量、零件12不合格数量、零件不合格导致成品不合格中零件12合格的数量
def assembly(part1, part2, product_defect_rate):
    product_success_q = 0#记录每次装配两零件均合格的成品数量
    product_defect_q = 0
    part1_defect_2 = 0#每次装配，零件1不合格的数量
    part2_defect_2 = 0#每次装配，零件2不合格的数量
    part1_success = 0#每次装配，一个零件不合格的成品中，零件1合格的数量
    part2_success = 0#每次装配，一个零件不合格的成品中，零件2合格的数量
    for i in range(min(len(part1), len(part2))):
        if part1[i] == 1 and part2[i] == 1:
            product_success_q += 1
        elif part1[i] == 1 and part2[i] == 0:
            product_defect_q += 1
            part1_success += 1
            part2_defect_2 += 1
        elif part1[i] == 0 and part2[i] == 1:
            product_defect_q += 1
            part1_defect_2 += 1
            part2_success += 1
        else:
            product_defect_q += 1
            part1_defect_2 += 1
            part2_defect_2 += 1
    assembly_defect = int(product_success_q * product_defect_rate)#每次装配导致的次品数量
    product_defect_q += assembly_defect#每次装配总次品数量
    return product_defect_q, assembly_defect, part1_defect_2, part2_defect_2, part1_success, part2_success

#对拆解过程进行模拟，更新零件列表
def disassembly(x1_1, x1_2, part1, part2, assembly_defect, part1_success, part2_success, part1_defect_2, part2_defect_2):
    if x1_1 == 0 and x1_2 == 0:
        new_part1 = [1] * (assembly_defect + part1_success) + [0] * part1_defect_2
        new_part2 = [1] * (assembly_defect + part2_success) + [0] * part2_defect_2
    elif x1_1 == 0 and x1_2 == 1:
        new_part1 = [1] * assembly_defect + [0] * part1_defect_2 + part1[len(part2):]
        new_part2 = [1] * (assembly_defect + part2_success)
    elif x1_1 == 1 and x1_2 == 0:
        new_part1 = [1] * (assembly_defect + part1_success)
        new_part2 = [1] * assembly_defect + [0] * part2_defect_2 + part2[len(part1):]
    else:
        new_part1 = [1] * assembly_defect
        new_part2 = [1] * assembly_defect
    random.shuffle(new_part1)
    random.shuffle(new_part2)
    return new_part1, new_part2

#模拟对成品检测、拆解
def test_disassembly(x1_1, x1_2, x2, x3, part1, part2, product_defect_rate, product_q, product_defect_q, assembly_defect, part1_success, part2_success, part1_defect_2, part2_defect_2,product_success_q):
    disassembly_q = product_defect_q#若拆卸，初始化拆卸总数量
    re_assembly_q = product_defect_q#若拆卸，初始化重新装配总数量
    if x2:  # 检测
        product_test_q = min(len(part1),len(part2))#若检测，初始化检测总数量
        replace_q = 0
        if x3:  # 拆解
            for i in range(3):
                new_part1, new_part2 = disassembly(x1_1, x1_2, part1, part2, assembly_defect, part1_success, part2_success, part1_defect_2, part2_defect_2)
                re_assembly_q += min(len(new_part1), len(new_part2))#累加每次拆解后，再装配数量
                product_test_q += min(len(new_part1), len(new_part2))#累加每次拆解后，再检测数量
                product_defect_q, assembly_defect, part1_defect_2, part2_defect_2, part1_success, part2_success = assembly(new_part1, new_part2, product_defect_rate)
                disassembly_q += product_defect_q#累加每次拆卸的成品数量
                product_success_q+=(min(len(new_part1), len(new_part2))-product_defect_q)#累加每次重新装配中，可以直接发给客户的产品数量
        else:  # 不拆解
            product_test_q = product_test_q#总检测数量不变，不新增销售数量
    else:  # 不检测
        product_test_q = 0
        replace_q = product_defect_q#若不检测，初始化客户调换数量
        if x3:  # 拆解
            for i in range(3):
                new_part1, new_part2 = disassembly(x1_1, x1_2, part1, part2, assembly_defect, part1_success, part2_success, part1_defect_2, part2_defect_2)
                re_assembly_q += min(len(new_part1), len(new_part2))#累加每次拆解后，再装配数量
                product_defect_q, assembly_defect, part1_defect_2, part2_defect_2, part1_success, part2_success = assembly(new_part1, new_part2, product_defect_rate)
                disassembly_q += product_defect_q#累加每次拆卸的成品数量
                replace_q += product_defect_q#累加每次装配后，客户调换数量
                product_success_q+=min(len(new_part1), len(new_part2))-product_defect_q
        else:  # 不拆解
            replace_q = product_defect_q#总退换数量不变，不新增销售数量
    return product_test_q, disassembly_q, re_assembly_q, replace_q,product_success_q

n = 5000

monte_carlo_simulation = 200

# Create separate counters for each situation
counters = [[0] * 16 for _ in range(6)]

def counter_c(x1_1, x1_2, x2, x3, situation_index):
    index = x1_1 * 8 + x1_2 * 4 + x2 * 2 + x3
    counters[situation_index][index] += 1

def initialize_defect_rates(mu, sigma):
    product_defect_rate = np.random.normal(mu, sigma)
    return np.clip(product_defect_rate, 0, 1)

# Rest of the functions remain the same
# ... (total_profit, first_assemblycost, assembly, disassembly, test_disassembly)

# Test data
situations = [
    {"part1_defect_rate": 0.1, "part1_cost": 4, "part1_test_cost": 2,
     "part2_defect_rate": 0.1, "part2_cost": 18, "part2_test_cost": 3,
     "product_defect_rate": 0.1, "assembly_cost": 6, "product_test_cost": 3,
     "market_price": 56, "replacement_loss": 6, "disassembly_cost": 5},

    {"part1_defect_rate": 0.2, "part1_cost": 4, "part1_test_cost": 2,
     "part2_defect_rate": 0.2, "part2_cost": 18, "part2_test_cost": 3,
     "product_defect_rate": 0.2, "assembly_cost": 6, "product_test_cost": 3,
     "market_price": 56, "replacement_loss": 6, "disassembly_cost": 5},

    {"part1_defect_rate": 0.1, "part1_cost": 4, "part1_test_cost": 2,
     "part2_defect_rate": 0.1, "part2_cost": 18, "part2_test_cost": 3,
     "product_defect_rate": 0.1, "assembly_cost": 6, "product_test_cost": 3,
     "market_price": 56, "replacement_loss": 30, "disassembly_cost": 5},

    {"part1_defect_rate": 0.2, "part1_cost": 4, "part1_test_cost": 1,
     "part2_defect_rate": 0.2, "part2_cost": 18, "part2_test_cost": 1,
     "product_defect_rate": 0.2, "assembly_cost": 6, "product_test_cost": 2,
     "market_price": 56, "replacement_loss": 30, "disassembly_cost": 5},

    {"part1_defect_rate": 0.1, "part1_cost": 4, "part1_test_cost": 8,
     "part2_defect_rate": 0.2, "part2_cost": 18, "part2_test_cost": 1,
     "product_defect_rate": 0.1, "assembly_cost": 6, "product_test_cost": 2,
     "market_price": 56, "replacement_loss": 10, "disassembly_cost": 5},

    {"part1_defect_rate": 0.05, "part1_cost": 4, "part1_test_cost": 2,
     "part2_defect_rate": 0.05, "part2_cost": 18, "part2_test_cost": 3,
     "product_defect_rate": 0.05, "assembly_cost": 6, "product_test_cost": 3,
     "market_price": 56, "replacement_loss": 10, "disassembly_cost": 40}
]

for i in range(monte_carlo_simulation):
    if i < 2:
        print(f"第{i+1}次模拟所得情况为:")
    
    for situation_index, situation in enumerate(situations):
        situation['part1_defect_rate'] = initialize_defect_rates(situation['part1_defect_rate'], 0.02)
        situation['part2_defect_rate'] = initialize_defect_rates(situation['part2_defect_rate'], 0.02)

        part1_defect = int(n * situation['part1_defect_rate'])
        part2_defect = int(n * situation['part2_defect_rate'])

        part1 = [1] * (n - part1_defect) + [0] * part1_defect
        part2 = [1] * (n - part2_defect) + [0] * part2_defect
        
        random.shuffle(part1)
        random.shuffle(part2)

        max_profit = float('-inf')
        best_strategy = None

        for x1_1 in range(2):
            for x1_2 in range(2):
                for x2 in range(2):
                    for x3 in range(2):
                        profit = total_profit(n, part1, part2, part1_defect, part2_defect, 
                                            situation['part1_cost'], situation['part2_cost'],
                                            situation['part1_test_cost'], situation['part2_test_cost'],
                                            situation['product_defect_rate'], situation['assembly_cost'],
                                            situation['product_test_cost'], situation['market_price'],
                                            situation['replacement_loss'], situation['disassembly_cost'],
                                            x1_1, x1_2, x2, x3)
                        if profit > max_profit:
                            max_profit = profit
                            best_strategy = (x1_1, x1_2, x2, x3)
                            
        counter_c(*best_strategy, situation_index)
        if i < 2:
            print(f"情况 {situation_index + 1}:")
            print(f"最大利润: {max_profit}")
            print(f"最佳策略: 检测零件1: {'是' if best_strategy[0] else '否'}, " 
                f"检测零件2: {'是' if best_strategy[1] else '否'}, "
                f"检测成品: {'是' if best_strategy[2] else '否'}, "
                f"拆解不合格品: {'是' if best_strategy[3] else '否'}")
            print()

print("Monte Carlo 模拟结果:")
for situation_index, counter in enumerate(counters):
    print(f"情况 {situation_index + 1}: {counter}")

第1次模拟所得情况为:
情况 1:
最大利润: 76072
最佳策略: 检测零件1: 是, 检测零件2: 是, 检测成品: 否, 拆解不合格品: 是

情况 2:
最大利润: 46051
最佳策略: 检测零件1: 是, 检测零件2: 是, 检测成品: 否, 拆解不合格品: 是

情况 3:
最大利润: 69611
最佳策略: 检测零件1: 是, 检测零件2: 否, 检测成品: 是, 拆解不合格品: 是

情况 4:
最大利润: 48046
最佳策略: 检测零件1: 是, 检测零件2: 是, 检测成品: 是, 拆解不合格品: 是

情况 5:
最大利润: 57899
最佳策略: 检测零件1: 否, 检测零件2: 是, 检测成品: 否, 拆解不合格品: 是

情况 6:
最大利润: 101324
最佳策略: 检测零件1: 否, 检测零件2: 否, 检测成品: 否, 拆解不合格品: 否

第2次模拟所得情况为:
情况 1:
最大利润: 65185
最佳策略: 检测零件1: 是, 检测零件2: 是, 检测成品: 否, 拆解不合格品: 是

情况 2:
最大利润: 45724
最佳策略: 检测零件1: 是, 检测零件2: 是, 检测成品: 否, 拆解不合格品: 是

情况 3:
最大利润: 72353
最佳策略: 检测零件1: 是, 检测零件2: 否, 检测成品: 是, 拆解不合格品: 是

情况 4:
最大利润: 42439
最佳策略: 检测零件1: 是, 检测零件2: 是, 检测成品: 是, 拆解不合格品: 是

情况 5:
最大利润: 44675
最佳策略: 检测零件1: 否, 检测零件2: 是, 检测成品: 是, 拆解不合格品: 是

情况 6:
最大利润: 102050
最佳策略: 检测零件1: 否, 检测零件2: 否, 检测成品: 否, 拆解不合格品: 否

Monte Carlo 模拟结果:
情况 1: [0, 5, 0, 0, 0, 22, 0, 0, 0, 119, 0, 0, 0, 54, 0, 0]
情况 2: [0, 0, 0, 0, 0, 52, 0, 0, 0, 0, 0, 0, 0, 148, 0, 0]
情况 3: [0, 0, 0, 0, 0, 5, 0, 50, 0, 6, 0, 94, 0, 45, 0, 0]
情况 4: [0, 0, 